In [1]:
import numpy as np
import pandas as pd

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, when, substring, concat, lit, to_date, date_format, sum as _sum

#### Start a simple Spark Session

In [3]:
spark = SparkSession.builder.appName('fraud_detection_data_overview').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/11 21:17:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/11 21:17:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Data Overview

In [4]:
df = spark.read.csv('credit_card_transactions-ibm_v2.csv', header=True, inferSchema=True)

In [5]:
df.printSchema()

root
 |-- User: integer (nullable = true)
 |-- Card: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Amount: string (nullable = true)
 |-- Use Chip: string (nullable = true)
 |-- Merchant Name: long (nullable = true)
 |-- Merchant City: string (nullable = true)
 |-- Merchant State: string (nullable = true)
 |-- Zip: double (nullable = true)
 |-- MCC: integer (nullable = true)
 |-- Errors?: string (nullable = true)
 |-- Is Fraud?: string (nullable = true)



In [6]:
df.describe().show()

24/06/11 21:17:23 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/06/11 21:17:25 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+-------+------------------+------------------+------------------+------------------+------------------+--------+-----------------+--------------------+-------------+--------------+------------------+-----------------+----------------+---------+
|summary|              User|              Card|              Year|             Month|               Day|  Amount|         Use Chip|       Merchant Name|Merchant City|Merchant State|               Zip|              MCC|         Errors?|Is Fraud?|
+-------+------------------+------------------+------------------+------------------+------------------+--------+-----------------+--------------------+-------------+--------------+------------------+-----------------+----------------+---------+
|  count|          24386900|          24386900|          24386900|          24386900|          24386900|24386900|         24386900|            24386900|     24386900|      21666079|          21508765|         24386900|          388431| 24386900|
|   mean|1001.01

#### Label Field Distribution

In [7]:
df.groupBy('Is Fraud?').count().orderBy('count').show()

+---------+--------+
|Is Fraud?|   count|
+---------+--------+
|      Yes|   29757|
|       No|24357143|
+---------+--------+



In [8]:
Fraud = df[df['Is Fraud?']=='Yes']

Normal = df[df['Is Fraud?']=='No']

In [9]:
fraud_dist = Fraud.count()/float(df.count())
fraud_dist*100

0.12202042900081601

The distribution of fraud cases reveals a highly imbalanced dataset, with the target class (fraud, Class==1) constituting only 0.122% of the total dataset. This raises a significant concern because most machine learning models are optimized to maximize accuracy. Consequently, classifying all transactions as normal would yield a substantial accuracy score, which is not beneficial for effective detection.

In [10]:
df.select("User").distinct().count()

2000

There are 2000 customers involved in these transactions.

### Data Preprocessing

In [11]:
# Count null values for each column
null_counts = df.select([_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Show the counts of null values
null_counts.show()

+----+----+----+-----+---+----+------+--------+-------------+-------------+--------------+-------+---+--------+---------+
|User|Card|Year|Month|Day|Time|Amount|Use Chip|Merchant Name|Merchant City|Merchant State|    Zip|MCC| Errors?|Is Fraud?|
+----+----+----+-----+---+----+------+--------+-------------+-------------+--------------+-------+---+--------+---------+
|   0|   0|   0|    0|  0|   0|     0|       0|            0|            0|       2720821|2878135|  0|23998469|        0|
+----+----+----+-----+---+----+------+--------+-------------+-------------+--------------+-------+---+--------+---------+



In [12]:
null_zip_df = df.filter(col('Zip').isNull())

# Group by 'Label' column and count occurrences
fraud_counts = null_zip_df.groupBy('Is Fraud?').count()

# Show the results
fraud_counts.show()

+---------+-------+
|Is Fraud?|  count|
+---------+-------+
|       No|2853283|
|      Yes|  24852|
+---------+-------+



The results of the analysis indicate a significant disparity in the occurrence of null values in the 'Zip' column between fraudulent and non-fraudulent transactions. 
Specifically, the data shows:<br><br>
24,852 fraudulent transactions (Label = 1) with null values in the 'Zip' column.<br>
2,853,283 non-fraudulent transactions (Label = 0) with null values in the 'Zip' column.<br><br>
This implies that a substantial percentage of the total null 'Zip' values are associated with fraudulent transactions. <br>
This pattern suggests a correlation between missing 'Zip' data and fraudulent activity, so needs a further investigation.

In [13]:
def preprocess(df):
    # df['Amount'] = df['Amount'].str.replace('$', "").astype(float)
    df = df.withColumn('Amount', regexp_replace(col('Amount'), '\\$', '').cast('float'))

    # df['Is Fraud?'] = df['Is Fraud?'].apply(lambda x: 0 if x=='No' else 1)
    df = df.withColumn('Is Fraud?', when(col('Is Fraud?') == "No", 0).otherwise(1))

    df = df.withColumn('Hour', substring('Time', 12, 2))
    df = df.withColumn('Minute', substring('Time', 15, 2))

    df = df.withColumn('Date', to_date(concat(col('Year'), lit('-'), col('Month'), lit('-'), col('Day')), 'yyyy-M-d'))

    df = df.withColumn('Day_of_Week', date_format('Date', 'EEEE'))

    # convert 'Hour' and 'Minute' fields into float data type
    df = df.withColumn('Hour', col('Hour').cast('float'))
    df = df.withColumn('Minute', col('Minute').cast('float'))
    df = df.drop('Zip')

    return df

In [14]:
df = preprocess(df)
df.show(2)

+----+----+----+-----+---+-------------------+------+-----------------+-------------------+-------------+--------------+----+-------+---------+----+------+----------+-----------+
|User|Card|Year|Month|Day|               Time|Amount|         Use Chip|      Merchant Name|Merchant City|Merchant State| MCC|Errors?|Is Fraud?|Hour|Minute|      Date|Day_of_Week|
+----+----+----+-----+---+-------------------+------+-----------------+-------------------+-------------+--------------+----+-------+---------+----+------+----------+-----------+
|   0|   0|2002|    9|  1|2024-06-11 06:21:00|134.09|Swipe Transaction|3527213246127876953|     La Verne|            CA|5300|   NULL|        0| 6.0|  21.0|2002-09-01|     Sunday|
|   0|   0|2002|    9|  1|2024-06-11 06:42:00| 38.48|Swipe Transaction|-727612092139916043|Monterey Park|            CA|5411|   NULL|        0| 6.0|  42.0|2002-09-01|     Sunday|
+----+----+----+-----+---+-------------------+------+-----------------+-------------------+-------------+

In [16]:
df.repartition(1).write.csv("df_EDA.csv", sep='|')